In [17]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle


In [18]:
df = pd.read_csv("gym_members_dataset.csv")

In [19]:
df.columns

Index(['Member_ID', 'Name', 'Age', 'Gender', 'Address', 'Phone_Number',
       'Membership_Type', 'Join_Date', 'Last_Visit_Date', 'Favorite_Exercise',
       'Avg_Workout_Duration_Min', 'Avg_Calories_Burned',
       'Total_Weight_Lifted_kg', 'Visits_Per_Month', 'Churn'],
      dtype='object')

In [20]:
#df = 
df.drop(['Member_ID', 'Name','Address', 'Phone_Number'],axis=1,inplace=True)

In [21]:
gender_label =LabelEncoder()
df['Gender']=gender_label.fit_transform(df['Gender'])
exercise_label =LabelEncoder()
df['Favorite_Exercise']=exercise_label.fit_transform(df['Favorite_Exercise'])
churn_label =LabelEncoder()
df['Churn']=churn_label.fit_transform(df['Churn'])

In [22]:
df['Membership_Type'].unique()

array(['Quarterly', 'Monthly', 'Yearly'], dtype=object)

In [23]:
from sklearn.preprocessing import OneHotEncoder
one = OneHotEncoder()
mem_one =one.fit_transform(df[['Membership_Type']])
mem_one_pd =pd.DataFrame(mem_one.toarray(), columns=one.get_feature_names_out(['Membership_Type']))
df = pd.concat([df, mem_one_pd], axis=1)


In [24]:
df.columns

Index(['Age', 'Gender', 'Membership_Type', 'Join_Date', 'Last_Visit_Date',
       'Favorite_Exercise', 'Avg_Workout_Duration_Min', 'Avg_Calories_Burned',
       'Total_Weight_Lifted_kg', 'Visits_Per_Month', 'Churn',
       'Membership_Type_Monthly', 'Membership_Type_Quarterly',
       'Membership_Type_Yearly'],
      dtype='object')

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 14 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Age                        137 non-null    float64
 1   Gender                     150 non-null    int32  
 2   Membership_Type            150 non-null    object 
 3   Join_Date                  141 non-null    object 
 4   Last_Visit_Date            150 non-null    object 
 5   Favorite_Exercise          150 non-null    int32  
 6   Avg_Workout_Duration_Min   150 non-null    int64  
 7   Avg_Calories_Burned        139 non-null    float64
 8   Total_Weight_Lifted_kg     142 non-null    float64
 9   Visits_Per_Month           138 non-null    float64
 10  Churn                      150 non-null    int32  
 11  Membership_Type_Monthly    150 non-null    float64
 12  Membership_Type_Quarterly  150 non-null    float64
 13  Membership_Type_Yearly     150 non-null    float64

In [26]:
df['Join_Date'] = pd.to_datetime(df['Join_Date'])
df['Last_Visit_Date'] = pd.to_datetime(df['Last_Visit_Date'])
df['Period'] = (df['Last_Visit_Date'] - df['Join_Date']).dt.days

In [27]:
df.head()

,Age,Gender,Membership_Type,Join_Date,Last_Visit_Date,Favorite_Exercise,Avg_Workout_Duration_Min,Avg_Calories_Burned,Total_Weight_Lifted_kg,Visits_Per_Month,Churn,Membership_Type_Monthly,Membership_Type_Quarterly,Membership_Type_Yearly,Period
0,19.0,1,Quarterly,2022-07-23,2022-11-03,3,93,214.0,13995.0,18.0,0,0.0,1.0,0.0,103.0
1,19.0,0,Monthly,2023-12-04,2024-02-14,4,37,436.0,4612.0,11.0,0,1.0,0.0,0.0,72.0
2,52.0,1,Quarterly,2024-09-13,2024-12-07,0,98,523.0,3124.0,4.0,1,0.0,1.0,0.0,85.0
3,52.0,1,Monthly,2024-06-05,2025-02-24,3,66,282.0,4586.0,24.0,0,1.0,0.0,0.0,264.0
4,32.0,1,Monthly,2022-06-15,2022-12-02,0,32,522.0,16353.0,18.0,0,1.0,0.0,0.0,170.0


In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 15 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Age                        137 non-null    float64       
 1   Gender                     150 non-null    int32         
 2   Membership_Type            150 non-null    object        
 3   Join_Date                  141 non-null    datetime64[ns]
 4   Last_Visit_Date            150 non-null    datetime64[ns]
 5   Favorite_Exercise          150 non-null    int32         
 6   Avg_Workout_Duration_Min   150 non-null    int64         
 7   Avg_Calories_Burned        139 non-null    float64       
 8   Total_Weight_Lifted_kg     142 non-null    float64       
 9   Visits_Per_Month           138 non-null    float64       
 10  Churn                      150 non-null    int32         
 11  Membership_Type_Monthly    150 non-null    float64       
 12  Membersh

In [29]:
df.drop(['Membership_Type', 'Last_Visit_Date','Join_Date'],axis=1,inplace=True)

In [30]:
df.head()

,Age,Gender,Favorite_Exercise,Avg_Workout_Duration_Min,Avg_Calories_Burned,Total_Weight_Lifted_kg,Visits_Per_Month,Churn,Membership_Type_Monthly,Membership_Type_Quarterly,Membership_Type_Yearly,Period
0,19.0,1,3,93,214.0,13995.0,18.0,0,0.0,1.0,0.0,103.0
1,19.0,0,4,37,436.0,4612.0,11.0,0,1.0,0.0,0.0,72.0
2,52.0,1,0,98,523.0,3124.0,4.0,1,0.0,1.0,0.0,85.0
3,52.0,1,3,66,282.0,4586.0,24.0,0,1.0,0.0,0.0,264.0
4,32.0,1,0,32,522.0,16353.0,18.0,0,1.0,0.0,0.0,170.0


In [33]:
# Save the Pickle File
with open("gender_label.pkl","wb") as f:
    pickle.dump(gender_label,f)
# Save the Pickle File
with open("exercise_label.pkl","wb") as f:
    pickle.dump(exercise_label,f)
with open("member_one.pkl","wb") as f:
    pickle.dump(one,f)

In [34]:
df

,Age,Gender,Favorite_Exercise,Avg_Workout_Duration_Min,Avg_Calories_Burned,Total_Weight_Lifted_kg,Visits_Per_Month,Churn,Membership_Type_Monthly,Membership_Type_Quarterly,Membership_Type_Yearly,Period
0,19.0,1,3,93,214.0,13995.0,18.0,0,0.0,1.0,0.0,103.0
1,19.0,0,4,37,436.0,4612.0,11.0,0,1.0,0.0,0.0,72.0
2,52.0,1,0,98,523.0,3124.0,4.0,1,0.0,1.0,0.0,85.0
3,52.0,1,3,66,282.0,4586.0,24.0,0,1.0,0.0,0.0,264.0
4,32.0,1,0,32,522.0,16353.0,18.0,0,1.0,0.0,0.0,170.0
...,...,...,...,...,...,...,...,...,...,...,...,...
145,54.0,1,3,64,733.0,6082.0,14.0,0,0.0,1.0,0.0,139.0
146,45.0,0,0,95,641.0,7375.0,12.0,0,1.0,0.0,0.0,51.0
147,54.0,1,2,80,471.0,6138.0,14.0,0,0.0,1.0,0.0,255.0
148,26.0,0,0,50,473.0,18336.0,4.0,1,1.0,0.0,0.0,326.0


# ANN

In [2]:
import tensorflow as tf

ModuleNotFoundError: No module named 'tensorflow'